In [1]:
%load_ext cython

# Python vs Cython classes

In [2]:
class Py_Student: # python class
    def __init__(self, name, weight, height):
        self.name = name
        self.weight = weight # in kg
        self.height = height # in m
    def get_bmi(self):
        return self.weight / (self.height * self.height)

In [3]:
%%cython

cdef class Cy_Student: # cython class
    cdef:
        str name
        public double weight # in kg
        double height # in m
    def __init__(self, name, weight, height):
        self.name = name
        self.weight = weight 
        self.height = height
    def get_bmi(self): # methods can be def, cdef or cpdef
        return self.weight / (self.height * self.height)

In [4]:
py_student = Py_Student("Hans", 110, 1.80) # create an instance from python class
cy_student = Cy_Student("Jon", 635, 1.85) # create an instance from cython class
print(py_student.get_bmi())
print(cy_student.get_bmi())

33.95061728395061
185.53688823959092


In [5]:
# compare speed of method calls from python
%timeit py_student.get_bmi()
%timeit cy_student.get_bmi()

254 ns ± 2.25 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
64.7 ns ± 0.314 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [6]:
# compare speed of creating objects
%timeit Py_Student("Kira", 55, 1.65)
%timeit Cy_Student("Lisa", 60, 1.75)

429 ns ± 9.05 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
138 ns ± 0.83 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Python places an object’s data inside an internal instance dictionary named \_\_dict__

In [7]:
print(py_student.__dict__)

{'name': 'Hans', 'weight': 110, 'height': 1.8}


In an extension type, the object’s data resides similar to C inside a struct

In [8]:
print(cy_student.weight)
# print(cy_student.__dict__) # fails --> object has no attribute '__dict__'

635.0


# \_\_cinit__ and \_\_dealloc__

In [9]:
%%cython

from libc.stdlib cimport realloc, free 

cdef class Vector:
    cdef:
        double *vec          # array of doubles
        public double[:] mv  # memory view, public --> can modify from python

    def __cinit__(self, long length): # constructor for any C-level initialization or memory allocation
        self.change_length(length)    # __cinit__ is called only once before __init__
        
    cpdef change_length(self, long new_length): # resize the vector
        self.vec = <double*>realloc(self.vec, new_length * sizeof(double)) # resize dynamic C-array
        self.mv = <double[:new_length]>self.vec # sets up a memory view for a dynamic C array
        
    def __dealloc__(self): # similar to a destructor in C++
        free(self.vec)  # free memory

In [10]:
# usage in python of custom Vector extension type
vec = Vector(2) # create a vector of length 2
vec.mv[0:2] = 1.0
print(list(vec.mv)) # [1.0, 1.0]
print(vec.mv.shape[0]) # 2
vec.change_length(4)  # change length of vector to 4
vec.mv[2:4] = 2.0 
print(list(vec.mv)) # [1.0, 1.0, 2.0, 2.0]

[1.0, 1.0]
2
[1.0, 1.0, 2.0, 2.0]
